## We take the hourly prices of bitcoin and other bitcoin companies and see any lead/lag relationships.

Bitcoin company tickers include:  MARA, RIOT, BWQ, CAN, GLXY, and SI

In [1]:
import ultron

In [3]:
ultron.reuters_cleaned_get_intraday_by_ric_frequency(ric='MARA', 
                                                     start_time='2020-01-01', 
                                                     end_time='2020-10-01', 
                                                     frequency='60')

,ric,utc_time,local_time,open,high,low,close,volume,no_trades,open_bid,high_bid,low_bid,close_bid,no_bids,open_ask,high_ask,low_ask,close_ask,no_asks


In [6]:
# First import the libraries that we need to use
import pandas as pd
import requests
import json

def fetch_daily_data(symbol):
    pair_split = symbol.split('/')  # symbol must be in format XXX/XXX ie. BTC/EUR
    symbol = pair_split[0] + '-' + pair_split[1]
    url = f'https://api.pro.coinbase.com/products/{symbol}/candles?granularity=86400'
    response = requests.get(url)
    if response.status_code == 200:  # check to make sure the response from server is good
        data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['unix'], unit='s')  # convert to a readable date
        data['vol_fiat'] = data['volume'] * data['close']      # multiply the BTC volume by closing price to approximate fiat volume

        # if we failed to get any data, print an error...otherwise write the file
        if data is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            return data
            data.to_csv(f'Coinbase_{pair_split[0] + pair_split[1]}_dailydata.csv', index=False)
    else:
        print("Did not receieve OK response from Coinbase API")

In [7]:
pair = 'BTC/USD'
btc = fetch_daily_data(pair)

In [8]:
btc

,unix,low,high,open,close,volume,date,vol_fiat
0,1609977600,36352.98,37823.00,36859.26,37420.20,15181.101361,2021-01-07,5.680798e+08
1,1609891200,33352.54,37000.00,34043.91,36859.26,45744.103200,2021-01-06,1.686094e+09
2,1609804800,29891.13,34499.67,32020.22,34030.64,42282.569200,2021-01-05,1.438903e+09
3,1609718400,27678.00,33666.99,33082.84,32019.99,46045.389685,2021-01-04,1.474373e+09
4,1609632000,32008.62,34810.00,32222.88,33080.66,36951.716506,2021-01-03,1.222387e+09
...,...,...,...,...,...,...,...,...
295,1584489600,5001.00,5452.82,5331.71,5413.64,37513.035235,2020-03-18,2.030821e+08
296,1584403200,4935.60,5569.44,5039.95,5331.71,44347.154873,2020-03-17,2.364462e+08
297,1584316800,4432.30,5350.30,5346.86,5037.61,80184.036852,2020-03-16,4.039359e+08
298,1584230400,5085.91,5985.00,5161.16,5345.35,43763.762368,2020-03-15,2.339326e+08


# Granger Causality Test

In [20]:
import yfinance as yf

In [23]:
mara = yf.Ticker("MARA")
hist = mara.history(period='max')
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2012-05-04,53.040001,119.599998,53.040001,119.599998,0,0,0.0
2012-05-07,119.599998,119.599998,119.599998,119.599998,0,0,0.0
2012-05-08,53.040001,104.000000,53.040001,104.000000,0,0,0.0
2012-05-09,104.000000,104.000000,104.000000,104.000000,0,0,0.0
2012-05-10,104.000000,104.000000,104.000000,104.000000,0,0,0.0
...,...,...,...,...,...,...,...
2020-12-30,12.560000,12.950000,10.710000,10.880000,55300400,0,0.0
2020-12-31,10.870000,11.610000,9.620000,10.440000,36596000,0,0.0
2021-01-04,12.480000,12.490000,10.530000,11.010000,35826600,0,0.0


In [11]:
from alpha_vantage.timeseries import TimeSeries

ALPHA_VANTAGE_API_KEY = 'W07IU2A8KEKYD2ZN'
ts = TimeSeries(key=ALPHA_VANTAGE_API_KEY, output_format='pandas')
intraday_data, data_info = ts.get_intraday('MARA', outputsize='full', interval='1min')
data_info

{'1. Information': 'Intraday (1min) open, high, low, close prices and volume',
 '2. Symbol': 'MARA',
 '3. Last Refreshed': '2021-01-06 20:00:00',
 '4. Interval': '1min',
 '5. Output Size': 'Full size',
 '6. Time Zone': 'US/Eastern'}

In [17]:
intraday_data

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2021-01-06 20:00:00,18.49,18.49,18.48,18.49,11558.0
2021-01-06 19:59:00,18.48,18.50,18.48,18.49,26211.0
2021-01-06 19:58:00,18.50,18.50,18.48,18.49,29557.0
2021-01-06 19:57:00,18.50,18.50,18.45,18.50,25222.0
2021-01-06 19:56:00,18.43,18.50,18.43,18.50,11430.0
...,...,...,...,...,...
2020-12-24 04:05:00,12.43,12.56,12.35,12.55,7083.0
2020-12-24 04:04:00,12.23,12.35,12.23,12.35,3160.0
2020-12-24 04:03:00,12.04,12.23,12.00,12.23,3239.0
